In [ ]:
from androguard.core.bytecodes import apk, dvm
from androguard.misc import AnalyzeAPK
import os

benign_apk_folder = "../benign-apks/"
malware_apk_folder = "../malware-apks/"
apk_folders = [benign_apk_folder, malware_apk_folder]

def extract_api_calls(apk_path):
    a, d, dx = AnalyzeAPK(apk_path)
    api_calls = set()

    for method in dx.get_methods():
        if method.is_external():
            continue
        for _, call, _ in method.get_xref_to():
            api_calls.add(call.get_class_name() + '->' + call.get_name())

    return list(api_calls)


In [ ]:
api_calls_dict = {}
for apk_folder in apk_folders:
    for filename in os.listdir(apk_folder):
        apk_path = os.path.join(apk_folder, filename)
        try:
            api_calls = extract_api_calls(apk_path)
        except:
            continue
        if filename.startswith('VirusShare'):
            file_hash = filename.split('_')[1].split('.')[0]
        else:
            file_hash = filename.replace('.apk', '')
        api_calls_dict[file_hash] = api_calls
        break

print(api_calls_dict)